In [11]:
%load_ext autoreload
%autoreload 2
%load_ext snoop

import itertools
import pandas
import csv
import torch
import numpy as np
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
# from allennlp.data.token_indexers.bert_tokenizers import PretrainedBertIndexer

from awesome_glue.task import Task
from awesome_glue.utils import AttackMetric
from awesome_glue.config import Config

from luna import ram_write, chunks
from allennlpx import allenutil

from luna import ram_read

from allennlpx.interpret.attackers.embedding_searcher import EmbeddingSearcher

from collections import Counter, defaultdict
from tqdm import tqdm

from copy import deepcopy
from typing import List

import numpy
import torch
import numpy as np
from allennlp.common.util import JsonDict, sanitize
from allennlp.data.fields import TextField
from allennlp.data.token_indexers import ELMoTokenCharactersIndexer, TokenCharactersIndexer
from allennlp.data.tokenizers import Token
from allennlp.modules.text_field_embedders.text_field_embedder import TextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from luna import cast_list, lazy_property
from allennlp.data.vocabulary import Vocabulary, DEFAULT_OOV_TOKEN

from allennlpx.interpret.attackers.attacker import Attacker, DEFAULT_IGNORE_TOKENS
from allennlpx.interpret.attackers.embedding_searcher import EmbeddingSearcher
from allennlpx import allenutil
from itertools import product
from collections import defaultdict
import random

from functools import lru_cache
from allennlp.data.tokenizers import SpacyTokenizer
from luna import time_record

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The snoop extension is already loaded. To reload it, use:
  %reload_ext snoop


In [8]:
config = Config()
# config.finetunable=True
task = Task(config) # type: Task
task.from_pretrained()
config

cache for SST-spacy exists
*** load SST-spacy from cache cost 2.5 seconds
cache for 2f2a46e51336b27a7262689b00016955 exists
*** load 2f2a46e51336b27a7262689b00016955 from cache cost 0.0124 seconds
Restore model from checkpoint saved/models/SST-lstm-glove@0.ckpt.


Basic Args:
	--task_id=SST
	--finetunable=True
	--arch=lstm
	--pretrain=glove
	--_model_name=SST-lstm-glove
	--mode=attack
	--adv_data=nogit/SST-bert.adv.tsv
	--aug_data=
	--attack_vectors=glove
	--attack_data_split=dev
	--attack_size=100
	--attack_gen_aug=False
	--attack_gen_adv=False
	--alchemist=False
	--seed=2
Deduced Args:
	--model_name=SST-lstm-glove
	--tokenizer=spacy

In [ ]:
task.vocab.get_index_to_token_vocabulary().values()

In [77]:
from dataclasses import dataclass

@dataclass
class La:
    pass
        
@dataclass
class La2(La):
    rank: str
    suit2: str
        

x = La()
y = La2("2,", "1")
print(x, y)
isinstance(y, La)

La() La2(rank='2,', suit2='1')


True

In [64]:
from allennlpx.interpret.attackers.pwws import PWWS
attacker = PWWS(task.predictor)
attacker.initialize()
sent = allenutil.as_sentence(task.dev_data[15])
print(sent)
print(task.predictor.predict(sent), '\n')
result = attacker.attack_from_json({"sent": sent},
                                   field_to_change="sent",
#                                    ignore_tokens=forbidden_words,
#                                    forbidden_tokens=forbidden_words,
                                   max_change_num_or_ratio=0.25,
                                   measure='euc',
                                   topk=300,
#                                                    search_num=256
                                  )
print(result)
print(allenutil.as_sentence(result['raw']))
print(allenutil.as_sentence(result['adv']))

the emotions are raw and will strike a nerve with anyone who 's ever had family trauma .
{'logits': [-1.4450730085372925, 1.5022544860839844], 'probs': [0.04986296594142914, 0.9501370787620544]} 

1 emotions emotion -0.0031312704 -0.005826175
2 are exist 0.3140344 -0.013517439
3 raw rude 0.8009925 -0.010275245
5 will leave 0.019747853 -0.014440477
6 strike assume 0.5191475 -0.03238195
7 a amp 0.11465877 -0.019205809
8 nerve spunk 0.031820893 -0.016161263
13 ever ever 0.0 -0.010089338
14 had sustain 0.03057003 -0.009660423
15 family category 0.032264113 -0.0156852
16 trauma hurt 0.04008025 -0.019684553
{'adv': ['the', 'emotions', 'are', 'rude', 'and', 'will', 'strike', 'a', 'nerve', 'with', 'anyone', 'who', "'s", 'ever', 'had', 'family', 'trauma', '.'], 'raw': ['the', 'emotions', 'are', 'raw', 'and', 'will', 'strike', 'a', 'nerve', 'with', 'anyone', 'who', "'s", 'ever', 'had', 'family', 'trauma', '.'], 'outputs': {'logits': [0.8706834316253662, -0.8706427812576294], 'probs': [0.85085546